In [13]:


import pandas as pd
import requests
import time
import random
import json
from typing import Tuple, Optional
import urllib.parse


CSV_FILE_PATH = 'archive/korean_drama.csv'
OUTPUT_FILE_PATH = 'korean_drama_with_genres.csv'
DRAMA_TITLE_COLUMN = 'drama_name'  




BASE_DELAY = 3  
SAVE_FREQUENCY = 10  



class GenreFinder:
    
    
    def __init__(self, tmdb_key=None, omdb_key=None):
        self.session = requests.Session()
        self.tmdb_key = tmdb_key
        self.omdb_key = omdb_key
        self.stats = {'tmdb': 0, 'omdb': 0, 'tvmaze': 0, 'predefined': 0, 'failed': 0}
        
    def find_genre(self, drama_name: str) -> Tuple[str, str]:
        """
        Find genre using multiple sources
        Returns: (genre, source)
        """
        # Strategy 1: Check predefined list first (instant)
        genre, source = self._check_predefined(drama_name)
        if genre:
            self.stats['predefined'] += 1
            return genre, source
        
        # Strategy 2: Try TMDB if API key available
        if self.tmdb_key:
            genre, source = self._search_tmdb(drama_name)
            if genre:
                self.stats['tmdb'] += 1
                return genre, source
            time.sleep(1)
        
        # Strategy 3: Try TVMaze (no API key needed, very reliable)
        genre, source = self._search_tvmaze(drama_name)
        if genre:
            self.stats['tvmaze'] += 1
            return genre, source
        time.sleep(1)
        
        # Strategy 4: Try OMDb if API key available
        if self.omdb_key:
            genre, source = self._search_omdb(drama_name)
            if genre:
                self.stats['omdb'] += 1
                return genre, source
        
        self.stats['failed'] += 1
        return "Not Found", "None"
    
    def _check_predefined(self, drama_name: str) -> Tuple[Optional[str], Optional[str]]:
        """Check against predefined genre list"""
        predefined = {
            # Add your known dramas here for instant lookup
            'd.p.': 'Action, Drama, Military',
            'dp': 'Action, Drama, Military',
            'celebrity': 'Drama, Thriller',
            'sing my crush': 'Romance, Music',
            'shadow detective': 'Crime, Mystery, Thriller',
            'to be honest': 'Drama, Romance',
            'blue temperature': 'Romance, Drama',
            'bloodhounds': 'Action, Crime, Drama',
            'destined with you': 'Romance, Fantasy',
            'good bad mother': 'Drama, Family',
            'king the land': 'Romance, Comedy',
            'move to heaven': 'Drama',
            'squid game': 'Action, Thriller, Drama',
            'crash landing on you': 'Romance, Comedy, Drama',
            'itaewon class': 'Drama',
            'vincenzo': 'Crime, Comedy, Drama',
            'hometown cha-cha-cha': 'Romance, Comedy',
            'extraordinary attorney woo': 'Drama, Comedy',
            'business proposal': 'Romance, Comedy',
            'alchemy of souls': 'Fantasy, Romance, Drama',
            'our blues': 'Drama, Romance',
            'twenty five twenty one': 'Drama, Romance',
            'all of us are dead': 'Horror, Thriller, Drama',
            'my name': 'Action, Crime, Thriller',
            'hellbound': 'Horror, Thriller, Fantasy',
            'the glory': 'Thriller, Drama',
            'sweet home': 'Horror, Thriller, Fantasy',
            'kingdom': 'Horror, Thriller, Historical',
        }
        
        drama_lower = drama_name.lower().strip()
        
        # Exact match
        if drama_lower in predefined:
            return predefined[drama_lower], "Predefined"
        
        # Partial match
        for key, genre in predefined.items():
            if key in drama_lower or drama_lower in key:
                return genre, "Predefined"
        
        return None, None
    
    def _search_tmdb(self, drama_name: str) -> Tuple[Optional[str], Optional[str]]:
        """Search The Movie Database API"""
        try:
            # Search for TV show
            search_url = "https://api.themoviedb.org/3/search/tv"
            params = {
                'api_key': self.tmdb_key,
                'query': drama_name,
                'language': 'en-US'
            }
            
            response = self.session.get(search_url, params=params, timeout=10)
            if response.status_code != 200:
                return None, None
            
            data = response.json()
            if not data.get('results'):
                return None, None
            
            # Get first result details
            tv_id = data['results'][0]['id']
            detail_url = f"https://api.themoviedb.org/3/tv/{tv_id}"
            detail_params = {'api_key': self.tmdb_key, 'language': 'en-US'}
            
            detail_response = self.session.get(detail_url, params=detail_params, timeout=10)
            if detail_response.status_code != 200:
                return None, None
            
            detail_data = detail_response.json()
            genres = [g['name'] for g in detail_data.get('genres', [])]
            
            if genres:
                return ', '.join(genres), "TMDB"
            
            return None, None
            
        except Exception as e:
            return None, None
    
    def _search_tvmaze(self, drama_name: str) -> Tuple[Optional[str], Optional[str]]:
        """Search TVMaze API (no key needed)"""
        try:
            search_url = "https://api.tvmaze.com/search/shows"
            params = {'q': drama_name}
            
            response = self.session.get(search_url, params=params, timeout=10)
            if response.status_code != 200:
                return None, None
            
            data = response.json()
            if not data:
                return None, None
            
            # Get first result
            show = data[0].get('show', {})
            genres = show.get('genres', [])
            
            if genres:
                return ', '.join(genres), "TVMaze"
            
            return None, None
            
        except Exception as e:
            return None, None
    
    def _search_omdb(self, drama_name: str) -> Tuple[Optional[str], Optional[str]]:
        """Search OMDb API"""
        try:
            url = "http://www.omdbapi.com/"
            params = {
                'apikey': self.omdb_key,
                't': drama_name,
                'type': 'series'
            }
            
            response = self.session.get(url, params=params, timeout=10)
            if response.status_code != 200:
                return None, None
            
            data = response.json()
            if data.get('Response') != 'True':
                return None, None
            
            genre = data.get('Genre')
            if genre and genre != 'N/A':
                return genre, "OMDb"
            
            return None, None
            
        except Exception as e:
            return None, None


def process_dramas():
    """Main processing function"""
    
    print("="*80)
    print("               KOREAN DRAMA GENRE SCRAPER v2.0")
    print("="*80)
    print()
    
    # Load CSV
    try:
        df = pd.read_csv(CSV_FILE_PATH)
        print(f"✓ Loaded CSV: {len(df)} dramas found")
    except FileNotFoundError:
        print(f"✗ ERROR: File '{CSV_FILE_PATH}' not found!")
        return
    except Exception as e:
        print(f"✗ ERROR loading CSV: {e}")
        return
    
    # Check if drama column exists
    if DRAMA_TITLE_COLUMN not in df.columns:
        print(f"\n✗ ERROR: Column '{DRAMA_TITLE_COLUMN}' not found!")
        print(f"Available columns: {', '.join(df.columns)}")
        return
    
    # Add genre columns if they don't exist
    if 'Genre' not in df.columns:
        df['Genre'] = None
    if 'Genre_Source' not in df.columns:
        df['Genre_Source'] = None
    
    # Create backup
    backup_path = CSV_FILE_PATH.replace('.csv', '_backup.csv')
    df.to_csv(backup_path, index=False)
    print(f"✓ Backup created: {backup_path}")
    
    # Check API keys
    print(f"\n📡 API Configuration:")
    print(f"   TMDB API: {'✓ Enabled' if TMDB_API_KEY else '✗ Not configured'}")
    print(f"   OMDb API: {'✓ Enabled' if OMDB_API_KEY else '✗ Not configured'}")
    print(f"   TVMaze API: ✓ Enabled (no key needed)")
    
    if not TMDB_API_KEY and not OMDB_API_KEY:
        print(f"\n💡 TIP: Get free API keys for better results:")
        print(f"   • TMDB: https://www.themoviedb.org/settings/api")
        print(f"   • OMDb: http://www.omdbapi.com/apikey.aspx")
    
    print("\n" + "="*80)
    print("Starting genre search...")
    print("="*80 + "\n")
    
    # Initialize finder
    finder = GenreFinder(tmdb_key=TMDB_API_KEY, omdb_key=OMDB_API_KEY)
    
    # Process each drama
    processed = 0
    skipped = 0
    found = 0
    not_found = 0
    
    for idx, row in df.iterrows():
        drama_name = str(row[DRAMA_TITLE_COLUMN]).strip()
        current_genre = row.get('Genre')
        
        # Skip if already has valid genre
        if pd.notna(current_genre) and current_genre not in ['Not Found', '', 'None']:
            print(f"[{idx+1}/{len(df)}] ⏭️  Skipping: {drama_name} (already has genre)")
            skipped += 1
            continue
        
        print(f"\n[{idx+1}/{len(df)}] 🔍 Searching: {drama_name}")
        
        # Find genre
        genre, source = finder.find_genre(drama_name)
        
        # Update dataframe
        df.loc[idx, 'Genre'] = genre
        df.loc[idx, 'Genre_Source'] = source
        
        # Update stats
        processed += 1
        if genre != "Not Found":
            found += 1
            print(f"         ✅ Found: {genre} [{source}]")
        else:
            not_found += 1
            print(f"         ❌ Not found")
        
        # Save progress periodically
        if processed % SAVE_FREQUENCY == 0:
            df.to_csv(OUTPUT_FILE_PATH, index=False, encoding='utf-8-sig')
            print(f"\n         💾 Progress saved ({processed} processed)")
            print(f"         📊 Found: {found} | Not Found: {not_found} | Skipped: {skipped}")
        
        # Delay between requests
        if idx < len(df) - 1:
            delay = BASE_DELAY + random.uniform(1, 3)
            time.sleep(delay)
    
    # Final save
    df.to_csv(OUTPUT_FILE_PATH, index=False, encoding='utf-8-sig')
    
    # Print final report
    print("\n" + "="*80)
    print("                          FINAL REPORT")
    print("="*80)
    print(f"\n📊 Processing Summary:")
    print(f"   Total dramas:        {len(df)}")
    print(f"   Processed:           {processed}")
    print(f"   Skipped:             {skipped}")
    print(f"   Genres found:        {found} ({found/(processed or 1)*100:.1f}%)")
    print(f"   Not found:           {not_found} ({not_found/(processed or 1)*100:.1f}%)")
    
    print(f"\n📡 Sources Used:")
    for source, count in finder.stats.items():
        if count > 0:
            emoji = "✓" if source != 'failed' else "✗"
            print(f"   {emoji} {source.upper()}: {count}")
    
    print(f"\n💾 Output saved to: {OUTPUT_FILE_PATH}")
    
    # Show sample of not found dramas
    if not_found > 0:
        not_found_dramas = df[df['Genre'] == 'Not Found'][DRAMA_TITLE_COLUMN].head(10).tolist()
        print(f"\n⚠️  Sample of dramas not found (first 10):")
        for drama in not_found_dramas:
            print(f"   • {drama}")
        print(f"\n   💡 Consider adding these to the predefined list")
    
    print("\n" + "="*80)
    print("✅ Process completed successfully!")
    print("="*80)


if __name__ == "__main__":
    try:
        process_dramas()
    except KeyboardInterrupt:
        print("\n\n⚠️  Process interrupted by user")
        print("Progress has been saved")
    except Exception as e:
        print(f"\n\n❌ Unexpected error: {e}")
        import traceback
        traceback.print_exc()

               KOREAN DRAMA GENRE SCRAPER v2.0

✓ Loaded CSV: 1752 dramas found
✓ Backup created: archive/korean_drama_backup.csv

📡 API Configuration:
   TMDB API: ✗ Not configured
   OMDb API: ✗ Not configured
   TVMaze API: ✓ Enabled (no key needed)

💡 TIP: Get free API keys for better results:
   • TMDB: https://www.themoviedb.org/settings/api
   • OMDb: http://www.omdbapi.com/apikey.aspx

Starting genre search...


[1/1752] 🔍 Searching: Sing My Crush
         ✅ Found: Romance, Music [Predefined]

[2/1752] 🔍 Searching: D.P. Season 2
         ✅ Found: Action, Drama, Military [Predefined]

[3/1752] 🔍 Searching: Shadow Detective Season 2
         ✅ Found: Crime, Mystery, Thriller [Predefined]

[4/1752] 🔍 Searching: To Be Honest
         ✅ Found: Drama, Romance [Predefined]

[5/1752] 🔍 Searching: Celebrity
         ✅ Found: Drama, Thriller [Predefined]

[6/1752] 🔍 Searching: Blue Temperature
         ✅ Found: Romance, Drama [Predefined]

[7/1752] 🔍 Searching: Numbers
         ✅ Found: 


[82/1752] 🔍 Searching: Island
         ✅ Found: Drama, Fantasy, Horror [TVMaze]

[83/1752] 🔍 Searching: Hit the Spot
         ✅ Found: Drama, Comedy, Science-Fiction [TVMaze]

[84/1752] 🔍 Searching: The Fabulous
         ✅ Found: Drama, Comedy, Romance [TVMaze]

[85/1752] 🔍 Searching: The Interest of Love
         ✅ Found: Drama, Romance [TVMaze]

[86/1752] 🔍 Searching: The New Employee
         ✅ Found: Comedy, Romance [TVMaze]

[87/1752] 🔍 Searching: Big Bet
         ✅ Found: Food [TVMaze]

[88/1752] 🔍 Searching: Trolley
         ✅ Found: Drama, Mystery [TVMaze]

[89/1752] 🔍 Searching: Missing: The Other Side Season 2
         ❌ Not found

[90/1752] 🔍 Searching: Red Balloon
         ✅ Found: Drama, Romance [TVMaze]

         💾 Progress saved (90 processed)
         📊 Found: 75 | Not Found: 15 | Skipped: 0

[91/1752] 🔍 Searching: The King of the Desert
         ✅ Found: Drama, Action [TVMaze]

[92/1752] 🔍 Searching: Turned on by You Season 2
         ❌ Not found

[93/1752] 🔍 Searchin


[172/1752] 🔍 Searching: Ongoing Loved One
         ❌ Not found

[173/1752] 🔍 Searching: Unicorn
         ✅ Found: Comedy [TVMaze]

[174/1752] 🔍 Searching: Good Job
         ✅ Found: Romance, Mystery [TVMaze]

[175/1752] 🔍 Searching: Love On Green
         ❌ Not found

[176/1752] 🔍 Searching: Girlfriend Project Day 1
         ✅ Found: Drama, Romance [TVMaze]

[177/1752] 🔍 Searching: Everyone, at Least Once
         ❌ Not found

[178/1752] 🔍 Searching: Anna: Extended Version
         ❌ Not found

[179/1752] 🔍 Searching: Stock Struck
         ✅ Found: Drama, Comedy [TVMaze]

[180/1752] 🔍 Searching: A Model Family
         ✅ Found: Drama, Crime, Thriller [TVMaze]

         💾 Progress saved (180 processed)
         📊 Found: 141 | Not Found: 39 | Skipped: 0

[181/1752] 🔍 Searching: If You Wish Upon Me
         ✅ Found: Drama [TVMaze]

[182/1752] 🔍 Searching: Cheonglywood
         ❌ Not found

[183/1752] 🔍 Searching: Hunted
         ✅ Found: Drama, Thriller [TVMaze]

[184/1752] 🔍 Searching: 


[264/1752] 🔍 Searching: Soundtrack #1
         ✅ Found: Drama, Romance [TVMaze]

[265/1752] 🔍 Searching: The King of Pigs
         ✅ Found: Drama, Thriller, Mystery [TVMaze]

[266/1752] 🔍 Searching: Two Universes
         ✅ Found: Comedy, Science-Fiction [TVMaze]

[267/1752] 🔍 Searching: International Student Haha Introduction
         ❌ Not found

[268/1752] 🔍 Searching: Blue of Winter
         ✅ Found: Romance, Sports [TVMaze]

[269/1752] 🔍 Searching: A Superior Day
         ✅ Found: Drama, Thriller [TVMaze]

[270/1752] 🔍 Searching: Kill Heel
         ✅ Found: Drama, Thriller [TVMaze]

         💾 Progress saved (270 processed)
         📊 Found: 207 | Not Found: 63 | Skipped: 0

[271/1752] 🔍 Searching: Crazy Love
         ✅ Found: Drama, Comedy, Romance [TVMaze]

[272/1752] 🔍 Searching: Ply Friends: Seoyeon University Class of '22
         ❌ Not found

[273/1752] 🔍 Searching: Life After One Minute
         ❌ Not found

[274/1752] 🔍 Searching: A Day of Dating
         ✅ Found: Drama, 


[354/1752] 🔍 Searching: Delivery
         ✅ Found: Fantasy [TVMaze]

[355/1752] 🔍 Searching: Adult Trainee
         ✅ Found: Drama, Comedy [TVMaze]

[356/1752] 🔍 Searching: Political Fever
         ❌ Not found

[357/1752] 🔍 Searching: Now, We Are Breaking Up
         ✅ Found: Drama, Romance [TVMaze]

[358/1752] 🔍 Searching: The Red Sleeve
         ✅ Found: Drama, Romance, History [TVMaze]

[359/1752] 🔍 Searching: Melancholia
         ✅ Found: Drama, Romance [TVMaze]

[360/1752] 🔍 Searching: Find Me if You Can
         ✅ Found: Romance, Thriller, Mystery [TVMaze]

         💾 Progress saved (360 processed)
         📊 Found: 266 | Not Found: 94 | Skipped: 0

[361/1752] 🔍 Searching: Not Found Love
         ❌ Not found

[362/1752] 🔍 Searching: Idol: The Coup
         ✅ Found: Drama, Music [TVMaze]

[363/1752] 🔍 Searching: Secret Royal Inspector & Joy
         ✅ Found: Drama, Comedy, History [TVMaze]

[364/1752] 🔍 Searching: Let Me Be Your Knight
         ✅ Found: Drama, Romance [TVMaze]

[

...


[1339/1752] 🔍 Searching: Moment
         ❌ Not found

[1340/1752] 🔍 Searching: Man Who Dies to Live
         ✅ Found: Drama, Family [TVMaze]

         💾 Progress saved (1340 processed)
         📊 Found: 914 | Not Found: 426 | Skipped: 0

[1341/1752] 🔍 Searching: Reunited Worlds
         ✅ Found: Comedy, Fantasy, Romance [TVMaze]

[1342/1752] 🔍 Searching: Hong Ik Super
         ✅ Found: Drama, Romance [TVMaze]

[1343/1752] 🔍 Searching: School 2017
         ✅ Found: Drama, Mystery [TVMaze]

[1344/1752] 🔍 Searching: The King in Love
         ✅ Found: Comedy, Romance, History [TVMaze]

[1345/1752] 🔍 Searching: Flat
         ✅ Found: Comedy [TVMaze]

[1346/1752] 🔍 Searching: Idol Fever
         ✅ Found: Drama, Comedy, Romance [TVMaze]

[1347/1752] 🔍 Searching: First Love Story
         ✅ Found: Drama, Romance [TVMaze]

[1348/1752] 🔍 Searching: The Bride of Habaek
         ✅ Found: Drama, Fantasy, Romance [TVMaze]

[1349/1752] 🔍 Searching: First Love Story Pilot
         ❌ Not found

[1350/


[1427/1752] 🔍 Searching: Saimdang, Light’s Diary
         ✅ Found: Drama, Romance, History [TVMaze]

[1428/1752] 🔍 Searching: Good Manager
         ✅ Found: Comedy [TVMaze]

[1429/1752] 🔍 Searching: Defendant
         ✅ Found: Thriller, Legal [TVMaze]

[1430/1752] 🔍 Searching: Moonlight Sonata
         ❌ Not found

         💾 Progress saved (1430 processed)
         📊 Found: 982 | Not Found: 448 | Skipped: 0

[1431/1752] 🔍 Searching: Missing 9
         ✅ Found: Drama, Thriller, Mystery [TVMaze]

[1432/1752] 🔍 Searching: A Touch on Your Cheek
         ❌ Not found

[1433/1752] 🔍 Searching: Secret Crushes Season 3
         ❌ Not found

[1434/1752] 🔍 Searching: Introverted Boss
         ✅ Found: Comedy, Romance [TVMaze]

[1435/1752] 🔍 Searching: Between Friends
         ❌ Not found

[1436/1752] 🔍 Searching: Voice
         ✅ Found: Crime, Thriller, Mystery [TVMaze]

[1437/1752] 🔍 Searching: Naked Fireman
         ✅ Found: Drama [TVMaze]

[1438/1752] 🔍 Searching: Happy Hostage
         ❌ No

         ❌ Not found

[1512/1752] 🔍 Searching: Romantic Boss
         ✅ Found: Drama, Romance [TVMaze]

[1513/1752] 🔍 Searching: Second to Last Love
         ✅ Found: Drama, Romance, Adult [TVMaze]

[1514/1752] 🔍 Searching: Spark
         ✅ Found: Drama, Music, Romance [TVMaze]

[1515/1752] 🔍 Searching: Women at a Game Company
         ✅ Found: Drama [TVMaze]

[1516/1752] 🔍 Searching: Tong: Memories
         ✅ Found: Drama, Action [TVMaze]

[1517/1752] 🔍 Searching: Age of Youth
         ✅ Found: Drama, Comedy, Romance [TVMaze]

[1518/1752] 🔍 Searching: Reply Pyeongchang, 100°F
         ❌ Not found

[1519/1752] 🔍 Searching: W
         ✅ Found: Crime, Mystery, Thriller [Predefined]

[1520/1752] 🔍 Searching: Horror Delivery Service
         ❌ Not found

         💾 Progress saved (1520 processed)
         📊 Found: 1052 | Not Found: 468 | Skipped: 0

[1521/1752] 🔍 Searching: Magic Cell Phone
         ❌ Not found

[1522/1752] 🔍 Searching: Bring It On, Ghost
         ✅ Found: Drama, Comedy, F


[1599/1752] 🔍 Searching: Signal
         ✅ Found: Crime, Thriller, Mystery [TVMaze]

[1600/1752] 🔍 Searching: One More Happy Ending
         ✅ Found: Drama, Comedy, Romance [TVMaze]

         💾 Progress saved (1600 processed)
         📊 Found: 1111 | Not Found: 489 | Skipped: 0

[1601/1752] 🔍 Searching: The Ordinary Life of Ms. 'O'!
         ❌ Not found

[1602/1752] 🔍 Searching: Only Boy
         ✅ Found: Drama, Romance [TVMaze]

[1603/1752] 🔍 Searching: Deux Yeoza
         ❌ Not found

[1604/1752] 🔍 Searching: Moorim School
         ✅ Found: Drama, Action, Fantasy [TVMaze]

[1605/1752] 🔍 Searching: Kiss Me If You Can
         ❌ Not found

[1606/1752] 🔍 Searching: The Cravings Season 2
         ❌ Not found

[1607/1752] 🔍 Searching: My Son-In-Law's Woman
         ✅ Found: Drama, Family, Romance [TVMaze]

[1608/1752] 🔍 Searching: Cheese in the Trap
         ✅ Found: Drama, Comedy, Romance [TVMaze]

[1609/1752] 🔍 Searching: Jang Young Shil
         ✅ Found: Drama, History [TVMaze]

[1610


[1685/1752] 🔍 Searching: My Mother Is a Daughter-In-Law
         ❌ Not found

[1686/1752] 🔍 Searching: Hello Monster
         ✅ Found: Romance, Thriller, Mystery [TVMaze]

[1687/1752] 🔍 Searching: My Beautiful Bride
         ✅ Found: Action, Crime, Mystery [TVMaze]

[1688/1752] 🔍 Searching: Romance Blue
         ✅ Found: Drama, Romance [TVMaze]

[1689/1752] 🔍 Searching: Girls' Love Story
         ✅ Found: Drama, Comedy, Romance [TVMaze]

[1690/1752] 🔍 Searching: Hidden Identity
         ✅ Found: Action, Crime, Thriller [TVMaze]

         💾 Progress saved (1690 processed)
         📊 Found: 1181 | Not Found: 509 | Skipped: 0

[1691/1752] 🔍 Searching: Great First Wives
         ❌ Not found

[1692/1752] 🔍 Searching: The Return of Hwang Geum Bok!
         ❌ Not found

[1693/1752] 🔍 Searching: Prince's Prince
         ✅ Found: Drama, Comedy [TVMaze]

[1694/1752] 🔍 Searching: High Society
         ❌ Not found

[1695/1752] 🔍 Searching: Oh! Grandmas
         ✅ Found: Drama, Family [TVMaze]

[1